**QHACK 2021**

Proposed project:  POSE (Parameterized Quantum Cirucit OSE) (SQOSE?  Some Quantum Optimal Subarchitecture Extraction)


Determining the optimal quantum circuit structure for a parameterized model (either a quantum classifier or regressor) involves searching over a large parameter space including: depth, number of qubits, entangler layer design, initialization, optimizer choice (and associated hyperparameters), embedding..

Since running exponentially large circuits are liable to run into barren plateau problems, there is a need for fast searching over circuit structures coupled with intialization choices.

POSE is a version of OSE (optimal subarchitecture extraction) that is adapted for Parameterized quantum circuits.  It is the adaption of the FPTAS algorithm introduced in arxiv.org/abs/2010.08512.

From the AWS blog post:
"We wanted to produce a version of BERT **(replace BERT with a PQC)** whose architectural parameters minimized its parameter size, inference speed, and error rate, and I wanted to show that these architectural parameters were optimal." 

Q1: Do quantum circuits have the $AB^nC$ property?
From the AWS blog post:
"Under some circumstances, the algorithm’s error rate has a similar correlation. Furthermore, whenever the “cost” associated with the first (call it A) and last layers of the network is lower than that of the middle layers (B), the runtime of a solution will not explode. I call all these assumptions the ABnC property, which BERT turns out to have." 

Q2: Are loss functions L-Lipschitz smooth?

Q3: Are quantum circuit gradients bounded (yes)


**References**
1. De Wynter (2020) "An Approximation Algorithm for Optimal Subarchitecture Extraction" [arxiv.org/abs/2010.08512]
2. De Wynter and Perry (2020) "Optimal Subarchitecture Extraction for BERT" [arxiv.org/abs/2010.10499]









In [ ]:
# what libraries do we need to import? 

### What is the $AB^nC$ property

In the paper by de Wynter (see Section 1.1) the $AB^nC$ property is defined as: 
>>...**the intermediate layers are considerably more expensive, both in terms of parameter size and operations required, than the input and output functions of the network**; *the corresponding optimization problem is L-Lipschitz smooth with bounded stochastic gradients*; and **the training procedure uses stochastic gradient descent as the optimizer**. We refer to these assumptions collectively as the strong $AB^nC$ property..

The smoothness of quantum optimization, and the bounded gradients have been shown in other papers (see the recent papers on circuit training  by Patrick Cole's group, Maria Schuld's group).

However, how important is the use of SGD as the optimizer?  As was seen in the QNG paper, SGD can give less that optimal training -- can we say that PQC training has these $AB^nC$ properties if we choose QNG as the optimizer (it is, a variant of SGD...).  Yes, this is addressed in Ref (1) in the Conclusions:

>The time complexity for Algorithm 1 can be further improved by using a strongly convex loss (Rakhlin et al., 2012) **or by employing optimizers other than SGD**. A popular choice of the latter is ADAM (Kingma and Ba, 2014), which is known to yield faster convergence guarantees for some convex optimization problems, but not necessarily all (Reddi et al., 2018). Regardless, it is also well-known that SGD finds solutions that are highly generalizable (Kleinberg et al., 2018; Dinh et al., 2017).

In [ ]:
# so a good demo could be the OSE found when the optimizer is fixed as SGD (using PennyLane's GradientDescentOptimizer) or when the optimizer is fixed as QNG 

Step 1: test for $AB^nC$, are teh intermediate layers more expensive (parmaeter size and operations needed) than the input/output functions.

**assuming we are just using the type of classifier built during the OpenHack Challenge**
* Input functions:  PennyLane Amplitude Encoding layers (how many parameters needed) [**note: with the AmplitudeEncoding layers we do not need to rescale the features to be in the range [-1,1]**]
* Output functions: weighted sum (n operations) over Z expectation values for each qubit in register (n expectation operators)
* Intermediate layers:  Rotation gates and Entangling layers (how many parameters needed)
* Optional:  can we also extend this to include the time to simulate the circuit?  *Or does that overlap with the inference time...*




In [ ]:
# import data from sklearn (define the dataset D)
from sklearn import datasets

n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

def get_train_and_test_sets(data,split_factor=0.8):
  """
  take a fraction of the data as training data, remaining fraction is testing data
  default is to use a 80/20 split of the data, randomly chosen
  """
  features = data[0]
  labels = data[1]
  train_indices = np.random.random(len(features),int(split_factor*len(features)),replace=False)
  X_train=features[train_indices]
  Y_train=labels[train_indices]

  test_indices = [x for x in range(len(features)) if x not in train_indices]
  X_test = features[test_indices]
  Y_test = features[test_indices]

  return X_train,Y_train,X_test,Y_test

In [ ]:
# rescale data s.t. input features are in [-1,1]  (double check PennyLane documentation, is this required?  I just do it out of habit)

### Define the W-coefficient for quantum circuits

From (arxiv.org/abs/2010.10499), the main goal of OSE is to find a architecture/neural network/ PQC (in our case) that optimizes 3 factors (these are Definitions 3,4,5 in Reference 1):

* the inference speed ($\mathcal{i}(\cdot)$).  In Ref(1) this is defined as the "total number of steps required to compute an output during the forward propagation step for a fixed-sized input".  In Ref(2) they use **the wall-clock time, on-CPU** (for fixed-input-size).
* the parameter size ($\mathcal{p}(\cdot)$).  In Ref (1) this is the number of trainable parameters (i.e. rotation gates + any classical weights) in the model (PQC)
* the error rate ($\mathcal{e}(\cdot)$) measured relative to **some set of expected values $Y$ and trained parameters $W^*$** In Ref (1) the error rate is defined by the accuracy of the classifier (mean number of labels that are correctly assigned).  After Definition 5 the author addresses one of the main concerns we had during our discussion today:

>The output of the function e(f(·;W;ξ),D) **is directly dependent on** the trained parameters, as well as (by extension) the hyperparameters and training procedure used to obtain them. In particular, we note that *the optimal weight set $W∗$ is the one that minimizes the error across all possible assignments in $W$ [weight space]*. Albeit obvious, this insight will allow us to prove the complexity bounds for this problem, as well as provide an approximation algorithm for it. **It is then clear that we are unable to pick an optimal architecture set $\Xi^*$ without evaluating the error of all the candidate networks $f(\cdot;W^*;\xi(j)), \: \forall \: \xi(j),\epsilon^* \in \Xi$ and $W^* \in W$**. The main results of our work, presented in Section 3, focus on obtaining a way to circumvent such a limitation.

The main way they circumvent this is through the use of "surrogate functions". 

**What is a surrogate function?**

The surrogate for inference speed $\hat{i}$ is already given:  just use the wall clock time.  There are a number of examples in the PennyLane tutorials that show how to use timeit to estimate the time needed to run circuits, etc.. 

The surrogate for parameter size is ... the parameter size (it's just a counting number)

The surrogate for the error rate is defined using a subset of the training data.  The training data is ($D$) and the surrogate error is defined using a set $B \subset D$. Again, from Ref (1):
> Following standard optimization techniques we then define the surrogate error as the empirical loss function over a subset $B \subset D$:
$$\hat{\mathcal{e}}(f(\cdot;W;\xi),B) = \frac{1}{|B|} \sum_{\langle x_i, y_i \rangle \in B \subset D}(f(x_i;W;\xi),y_i).$$

The surrogate functions are all put together into the W-coefficient:
$$ \mathcal{W}(a,b) = \left(\frac{p(a) - p(b)}{p(a)}\right) \left(\frac{\hat{i}(a) - \hat{i}(b)}{\hat{i}(a)}\right) \left(\frac{1}{\hat{e}(b)}\right)$$ 

In [ ]:
# What are the steps needed for 

***# Start adapting Algorithm 1 in Ref (1) to quantum circuits***

(I am not going to reproduce the pseudocode here but these are the main concepts that need to get defined in terms of PQCs)


1.   Architecture (f): Quantum classifier (what rotation gates? entangling layers?)
2.   Dataset (D): 
3.   Weights space (W)
4.   Search space ($\Xi$)
5.   Hyperparameter set ($\Theta$)
6.   Interval size $1 \leq \epsilon \leq |\Xi|$
7.   Maximum training steps $s > 0$
8.   Selected loss $\ell$


The search space and the hyperparameter set $\Xi, \Theta$ will be the most relevant:

For the search space in Ref (1) they use (see Definition 2, with some subsitutions..):
>The search space Ξ = {ξ(1), . . . ξ(m)} is a finite set of valid assignments for the **architectural parameters** of a [parameterized quantum circuit] $\mathcal{U}(x; \theta ; \xi(i))$, such that for every assignment $\xi(i)$, the corresponding weight assignment set $\theta(i) \in W$ is non-empty. 

Examples of architectural parmaeters are things like:  number of hidden units, number of layers, and other countable parameters. Examples of architectural parameters in Ref (2) are: Layers, number of Attention heads, hidden unit size, intermediate layer size.  

My interpretation of these examples is that the architectural parameters have to be given in numeric value -- not for example using descriptive things, so the choice of 'ReLU' or 'softmax' or 'tanh' etc aren't architectural parameters.

For a quantum circuit what can we use:
* Depth of circuit < -- > number of layers
* number of CNOTs in the entangling layer? (but then how to distinguish between a layer that connects all qubits in a chain vs. a tree... they would have the same number of CNOTs... max degree of connectivity needed?
* Number of RX gates in rotation layer
* Number of RY gates in rotation layer
* Number of RZ gates in rotation layer
* Do we count the weights used in the final weighted sum as part of the cirucit, or as part of the output function??

* ??????


In [ ]:
# as an example, how can I write down an ordered tuple of numbers to describe the multiclass classifier we trained in the OpenHack Challenge

In [ ]:
# maybe as an initial test we keep the entangling layout fixed (this is our "fixed architecture") and the remaining parameters are now
# D: number of layers, number of RX gates in rotation layer, number of RY gates in rotation layer, number of RZ gates? 

## Finding a maximum point

In Ref(2) the OSE for BERT is found w/r/t a known, well-trained, high performing version of BERT (RoBERTa-large) and in Ref(1) the second line of the pseudo-code states "Find a maximum point"

Maximum point (T) is defined in Definition 8 of Ref(1): it is the set of architecture parameters that is largest (most parameters, longest inference time) and the important distinction that: "Note that the maximum point on $\Xi$ **is not an optimal point**–our goal is to minimize all three functions, and $T (\cdot; W_T^{∗} ;\xi_T )$ does not have a known error rate on D." [notice that in the W-coefficient there is only dependence on $\hat{e}(b)$..]

Basically we need to set up some baseline values for $p(T), i(T)$ (line 3 in pseudocode in Ref (1) ).  In Ref(2) they used the parameter size, and inference time associated with (RoBERTa-large).  For PQCs there are not many established benchmark designs, so we have some flexibility here.

In [ ]:
# for the initial test above, what if the maximum number of RX, RY and RZ gates in each rotation layer is 1 and the max number of layers is 100; so that the max point is
# T = (100,1,1,1)